In [1]:
from importlib import reload
import utils
import models #_gpu as models
import numpy as np
from time import time
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger # maybe more
import json

Using TensorFlow backend.


In [23]:
def run_kumar_model(it_no):


    BATCH_SIZE = 256
    EPOCHS = 10 # it seemed to converge by this point anyway

    # choose hyperparameters
    #num_layers = np.random.randint(1,5) # lstm layers
    #hidden_state_size = np.random.randint(100,400)
    #dropout_rate = round(np.random.uniform(0,1,size=None), 3)
    #embed_vec = np.random.choice(['glove200', 'glove300', 'numberbatch'])
    #sequence_length = 5
    bidirectional = True
    stateful = False
    trainable_embed = True
    
    num_layers = 1
    hidden_state_size = 300
    dropout_rate = 0.2
    embed_vec = 'glove300'
    sequence_length = 7

    ## start the timer
    start = time()

    # load the corpus
    corpus = utils.AMI_Corpus(seed = 75, embed_vec = embed_vec)

    bi_lstm = models.BiLSTMCRF(corpus,
                               batch_size = BATCH_SIZE,
                               sequence_length = sequence_length,
                               num_layers = num_layers,
                               dropout_rate = dropout_rate,
                               hidden_state_size = hidden_state_size,
                               stateful = stateful, bidirectional = bidirectional, 
                               trainable_embed = trainable_embed)

    bi_lstm.compile()

    ug_train = utils.UtteranceGenerator(corpus, "train", batch_size = BATCH_SIZE, sequence_length = sequence_length, conv = True, kumar = True)
    ug_val = utils.UtteranceGenerator(corpus, "val", batch_size = BATCH_SIZE, sequence_length = sequence_length, conv = True, kumar = True)

    # create keras callbacks
    es = EarlyStopping(monitor='val_loss', patience=5, verbose=0)

    #right_now = datetime.now().isoformat() # timestamp
    csv_logger = CSVLogger('logs/kumar_history_' + str(it_no) + ".csv") # log epochs in case I want to look back later

    # note to self, maybe change validation_steps and validation_freq
    history = bi_lstm.model.fit_generator(ug_train, epochs=EPOCHS, verbose=1, callbacks=[es, csv_logger],
                                          validation_data=ug_val)

    results = {"num_layers":num_layers, "hidden_state_size":hidden_state_size,"dropout_rate":dropout_rate, "sequence_length":sequence_length,"embed_vec":embed_vec, "acc":history.history['crf_viterbi_accuracy'][-5:],
               "val_acc":history.history['val_crf_viterbi_accuracy'][-5:], "loss":history.history['loss'][-5:],
               "val_loss":history.history['val_loss'][-5:], "time":time() - start,"trainable_embed":trainable_embed,
               "stateful":stateful, "bidirectional":bidirectional}
    
    results = {str(k):str(v) for k,v in results.items()}

    with open("logs/kumar_params_" + str(it_no) + ".json", "w") as f:
        json.dump(results, f)
    
    print("-------------------")

    
    return results

In [8]:
reload(utils)
reload(models)
for i in range(2,5):
    reults = run_kumar_model(it_no = i)

Loading corpus from /home/jake_miller/final/nlp-final-project/data
Begin corpus post-processing ...
Splitting corpus into training and test ...
Creating vocabulary from training set ...
Found 8435 unique words.
Building initial embedding matrix ...
(8437, 300)
loading pretrained vectors from numberbatch-en.txt
Epoch 1/10
311/311 [==============================] - 129s 414ms/step - loss: 2.0635 - crf_viterbi_accuracy: 0.2822 - val_loss: 1.8820 - val_crf_viterbi_accuracy: 0.3306
Epoch 2/10
311/311 [==============================] - 114s 367ms/step - loss: 1.7461 - crf_viterbi_accuracy: 0.3754 - val_loss: 1.6646 - val_crf_viterbi_accuracy: 0.4221
Epoch 3/10
311/311 [==============================] - 114s 366ms/step - loss: 1.4837 - crf_viterbi_accuracy: 0.4664 - val_loss: 1.3633 - val_crf_viterbi_accuracy: 0.5224
Epoch 4/10
311/311 [==============================] - 114s 368ms/step - loss: 1.3235 - crf_viterbi_accuracy: 0.5260 - val_loss: 1.2620 - val_crf_viterbi_accuracy: 0.5499
Epoch 5/

In [ ]:
# next, sequence = 6 ...

In [22]:
reults = run_kumar_model(it_no = 5)

Loading corpus from /home/jake_miller/final/nlp-final-project/data
Begin corpus post-processing ...
Splitting corpus into training and test ...
Creating vocabulary from training set ...
Found 8435 unique words.
Building initial embedding matrix ...
(8437, 300)
loading pretrained vectors from glove.6B.300d.txt
Epoch 1/10
311/311 [==============================] - 67s 217ms/step - loss: 1.5900 - crf_viterbi_accuracy: 0.4621 - val_loss: 1.2778 - val_crf_viterbi_accuracy: 0.5694
Epoch 2/10
311/311 [==============================] - 47s 151ms/step - loss: 1.2235 - crf_viterbi_accuracy: 0.5730 - val_loss: 1.1588 - val_crf_viterbi_accuracy: 0.5905
Epoch 3/10
311/311 [==============================] - 47s 152ms/step - loss: 1.1103 - crf_viterbi_accuracy: 0.5947 - val_loss: 1.0639 - val_crf_viterbi_accuracy: 0.6094
Epoch 4/10
311/311 [==============================] - 47s 152ms/step - loss: 1.0131 - crf_viterbi_accuracy: 0.6173 - val_loss: 0.9926 - val_crf_viterbi_accuracy: 0.6200
Epoch 5/10
31

In [24]:
# now, for sequence length 7 ...
reults = run_kumar_model(it_no = 6)

Loading corpus from /home/jake_miller/final/nlp-final-project/data
Begin corpus post-processing ...
Splitting corpus into training and test ...
Creating vocabulary from training set ...
Found 8435 unique words.
Building initial embedding matrix ...
(8437, 300)
loading pretrained vectors from glove.6B.300d.txt
Epoch 1/10
311/311 [==============================] - 78s 251ms/step - loss: 1.5766 - crf_viterbi_accuracy: 0.4652 - val_loss: 1.2613 - val_crf_viterbi_accuracy: 0.5749
Epoch 2/10
311/311 [==============================] - 55s 176ms/step - loss: 1.2159 - crf_viterbi_accuracy: 0.5747 - val_loss: 1.1556 - val_crf_viterbi_accuracy: 0.5907
Epoch 3/10
311/311 [==============================] - 55s 177ms/step - loss: 1.1059 - crf_viterbi_accuracy: 0.5983 - val_loss: 1.0585 - val_crf_viterbi_accuracy: 0.6132
Epoch 4/10
311/311 [==============================] - 55s 176ms/step - loss: 1.0145 - crf_viterbi_accuracy: 0.6213 - val_loss: 0.9935 - val_crf_viterbi_accuracy: 0.6274
Epoch 5/10
31